In [ ]:
# !pip install openai nbformat astor pyperclip nbconvert markdown


     -------------------------------------- 107.4/107.4 kB 3.1 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
import os
import nbformat
from nbconvert import PythonExporter
from openai import OpenAI
import datetime
import pyperclip
import textwrap
import markdown

In [27]:
# 🔑 1. API Key 불러오기
with open("./key/.openai_api_key") as f:
    api_key = f.read().strip()
client = OpenAI(api_key=api_key)

In [51]:
# 📂 2. 코드 + 마크다운 불러오기 함수
def load_code(file_path):
    if file_path.endswith(".ipynb"):
        # Jupyter Notebook 파일 읽기
        with open(file_path, encoding="utf-8") as f:
            nb = nbformat.read(f, as_version=4)

        # nbconvert의 PythonExporter를 활용해 코드+마크다운 변환
        exporter = PythonExporter()
        source, _ = exporter.from_notebook_node(nb)
        return source

    elif file_path.endswith(".py"):
        # 일반 파이썬 파일은 그대로 읽기
        return open(file_path, encoding="utf-8").read()

    else:
        raise ValueError("지원하지 않는 파일 형식입니다.")

# ✅ 테스트
code = load_code("./data/ex01_LangGraph_기초.ipynb")
print(code[:500]) 

#!/usr/bin/env python
# coding: utf-8

# ### 📘 LangChain · LangSmith · LangGraph · Agent 비교표
# | 구분        | **LangChain**             | **LangSmith**          | **LangGraph**                         | **Agent**                            |
# | --------- | ------------------------- | ---------------------- | ------------------------------------- | ------------------------------------ |
# | **정의**    | LLM 앱 **개발 프레임워크**        | LLM 앱 **관찰·평가·운영 플랫폼** | LLM 앱을 **그래프 기반으로 설계**하는 실행 엔진        | LL


In [52]:
# 📝 3. 템플릿
def make_prompt(code):
    template = textwrap.dedent(f"""
    아래는 학습 코드와 마크다운 설명입니다.  
    이를 분석해서 반드시 원본 도메인을 반영한 학습 요약을 작성하세요.  

    조건:
    - "핵심 코드"는 반드시 원본 코드에서 중요한 부분만 발췌(최대 10줄).
    - 새로운 예제를 만들지 말고 원본 코드를 기반으로 작성.

    --- 코드와 마크다운 시작 ---
    {code}
    --- 코드와 마크다운 끝 ---

    # 📒 학습 요약
    ## 1. 오늘 배운 주제
    - (한 줄 요약)

    ## 2. 핵심 코드
    ```python
    (중요한 코드 발췌, 최대 10줄)
    ```

    ## 3. 코드 해설
    - (주석을 기반으로 설명)

    ## 4. 언제 & 왜
    - 언제? ...
    - 왜? ...

    ## 5. 실무 적용 아이디어
    - ...

    ## 6. 확장 질문
    - ...
    """)
    return template

In [59]:
# 🚀 4. 실행
file_path = "./data/ex01_LangGraph_기초.ipynb"
code = load_code(file_path)
prompt = make_prompt(code)

messages = [
    {
        "role": "system",
        "content": (
            "너는 학습 요약 도우미다. "
            "사용자가 제공한 코드와 마크다운만 분석해라. "
            "절대로 임의의 예제를 만들어내지 말고, "
        )
    },
    {"role": "user", "content": prompt}
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=0,
    messages=messages
)
summary = response.choices[0].message.content
print(summary)

# 📒 학습 요약
## 1. 오늘 배운 주제
- LangGraph를 활용한 LLM 기반의 에이전트 및 챗봇 구축 방법

## 2. 핵심 코드
```python
# 1. 상태 정의
class State(TypedDict):
    messages: Annotated[list, add_messages]

# 2. 웹 검색도구 정의
from langchain_teddynote.tools.tavily import TavilySearch
tool_tavily = TavilySearch(max_results=2)

# 3. 챗봇 노드 정의
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# 4. 그래프 생성 및 노드 추가
graph_builder = StateGraph(State)
graph_builder.add_node('chatbot', chatbot)

# 5. 엣지 추가
graph_builder.add_edge(START, 'chatbot')
```

## 3. 코드 해설
- **상태 정의**: `State` 클래스를 정의하여 메시지 리스트를 포함하는 타입을 설정한다.
- **웹 검색 도구 정의**: `TavilySearch`를 사용하여 웹 검색 기능을 추가한다.
- **챗봇 노드 정의**: 사용자의 메시지를 받아 LLM을 호출하여 응답을 생성하는 함수를 정의한다.
- **그래프 생성 및 노드 추가**: `StateGraph` 객체를 생성하고 챗봇 노드를 추가한다.
- **엣지 추가**: 그래프의 시작점과 챗봇 노드를 연결하는 엣지를 추가한다.

## 4. 언제 & 왜
- 언제? LLM 기반의 챗봇이나 에이전트를 구축할 때.
- 왜? 사용자의 요청에 따라 동적으로 도구를 호출하고, 상태를 관리하여 보다 유연한 응답을 제공하기 위해.

## 5. 실무 적용 아이디어
- 고객 지원 챗봇에 LangGraph를 적용하여 사용자의 질문에 따라 웹 검색

In [60]:
# 📂 5. 저장 + 클립보드
today = datetime.date.today().strftime("%Y-%m-%d")
summary_file = f"learning_summary_{today}.md"

with open(summary_file, "w", encoding="utf-8") as f:
    f.write(summary)

pyperclip.copy(summary)
print(f"✅ 학습 요약 저장 완료: {summary_file}")
print("📋 클립보드에도 복사되었습니다. (Notion에 붙여넣으세요)")


✅ 학습 요약 저장 완료: learning_summary_2025-09-18.md
📋 클립보드에도 복사되었습니다. (Notion에 붙여넣으세요)


In [ ]:
# 📂 LLM 인포그래픽 생성 함수
def code_to_card_infographic_llm(code_str, output_file, title="학습 인포그래픽"):
    # 카드 한 장 예시 (구조만 참고)
    example_card = """
    <div class="card data">
      <div class="icon">📂</div>
      <div class="title">[단계명]</div>
      <div class="desc">[설명]</div>
      <div class="keywords">
        <div class="tag">[키워드]</div>
      </div>
    </div>
    """

    prompt = f"""
    아래는 학습 코드입니다. 코드를 분석해서 학습 과정의 주요 단계를 뽑아 인포그래픽을 만들어주세요.

    조건:
    - 반드시 아래 카드 예시와 동일한 구조/스타일을 사용하세요.
    - 예시 내용은 그대로 쓰지 말고, 코드에서 의미를 뽑아 새로운 단계명/설명/키워드로 채우세요.
    - 각 카드에는 (아이콘, 단계명, 설명, 키워드 태그)가 있어야 합니다.
    - 키워드 태그는 코드에 실제 등장한 함수명/클래스명/메서드명을 넣으세요.
    - 여러 카드를 grid 레이아웃 안에 넣어주세요.
    - 전체 HTML 문서 형태로 출력하세요 (<html> ~ </html> 포함).
    - 제목은 "{title}" 로 작성하세요.

    --- 카드 예시 (구조만 참고) ---
    {example_card}
    --- 카드 예시 끝 ---

    --- 코드 시작 ---
    {code_str}
    --- 코드 끝 ---
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system", "content": "너는 HTML 인포그래픽 디자이너다."},
            {"role": "user", "content": prompt}
        ]
    )

    html_content = response.choices[0].message.content

    with open(output_file, "w", encoding="utf-8") as f:
        f.write(html_content)

    print(f"✅ 카드형 인포그래픽 HTML 생성 완료: {output_file}")
    return html_content

# 🚀 실행
if __name__ == "__main__":
    file_path = "./data/ex01_LangGraph_기초.ipynb"  # 분석할 파일
    code = load_code(file_path)

    today = datetime.date.today().strftime("%Y-%m-%d")
    html_file = f"card_infographic_from_code_{today}.html"

    code_to_card_infographic_llm(
        code,
        html_file,
        title="LangGraph 기초 학습 인포그래픽"
    )


✅ 카드형 인포그래픽 HTML 생성 완료: card_infographic_from_code_2025-09-18.html
